---
## Sección 1. Configuración inicial

In [26]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers
from tensorflow.keras.datasets import mnist

import numpy as np
import matplotlib.pyplot as plt

import mlflow
import dagshub
import time

print(f"TensorFlow: {tf.__version__}")
print(f"MLflow: {mlflow.__version__}")

TensorFlow: 2.20.0
MLflow: 3.6.0


In [27]:
# Configurar DagsHub
dagshub.init(repo_owner='404brainnotfound-ai', repo_name='RNA', mlflow=True)

print(f"Tracking URI: {mlflow.get_tracking_uri()}")

Initialized MLflow to track repo "404brainnotfound-ai/RNA"

Repository 404brainnotfound-ai/RNA initialized!

Tracking URI: https://dagshub.com/404brainnotfound-ai/RNA.mlflow


In [28]:
# Verificar disponibilidad de GPU y listar dispositivos físicos
gpus = tf.config.list_physical_devices('GPU')
print(f"GPUs disponibles: {len(gpus)}")

if gpus:
    for i, gpu in enumerate(gpus):
        print(f"  GPU {i}: {gpu}")
    
    # Configurar crecimiento de memoria (evita que TensorFlow reserve toda la VRAM)
    tf.config.experimental.set_memory_growth(gpu, True)
    
    # Información adicional sobre la GPU
    print("\nDetalles de GPU:")
    from tensorflow.python.client import device_lib
    local_devices = device_lib.list_local_devices()
    for device in local_devices:
        if device.device_type == 'GPU':
            print(f"  Nombre: {device.physical_device_desc}")
else:
    print("No se detectó ninguna GPU. El entrenamiento usará CPU.")

GPUs disponibles: 1
  GPU 0: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

Detalles de GPU:
  Nombre: device: 0, name: NVIDIA GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1


I0000 00:00:1763956731.063360   27461 gpu_device.cc:2020] Created device /device:GPU:0 with 2857 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1


In [29]:
# Cargamos lso datos

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Aplanar y normalizar
x_train = x_train.reshape(60000, 784).astype('float32') / 255.0
x_test = x_test.reshape(10000, 784).astype('float32') / 255.0

# Onehot encoding
num_classes = 10
y_train_cat = keras.utils.to_categorical(y_train, num_classes)
y_test_cat = keras.utils.to_categorical(y_test, num_classes)

In [30]:
def train_and_log(model, run_name, params, epochs=15, batch_size=10):
    """Entrena un modelo y registra en MLflow."""
    
    with mlflow.start_run(run_name=run_name):
        # Registrar parámetros
        mlflow.log_params(params)
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)
        
        # Entrenar
        start_time = time.time()
        history = model.fit(
            x_train, y_train_cat,
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            validation_data=(x_test, y_test_cat))
        
        training_time = time.time() - start_time
        
        # Registrar métricas finales
        mlflow.log_metric("training_time", training_time)
        mlflow.log_metric("train_accuracy", history.history['accuracy'][-1])
        mlflow.log_metric("val_accuracy", history.history['val_accuracy'][-1])
        mlflow.log_metric("train_loss", history.history['loss'][-1])
        mlflow.log_metric("val_loss", history.history['val_loss'][-1])
        
        # Guardar modelo
        mlflow.keras.log_model(model, "model")
        
        # Resumen
        print(f"\n{run_name}: Val Acc = {history.history['val_accuracy'][-1]:.4f} | Tiempo = {training_time:.2f}s\n")
        
        return history

---
# Sección 2: Experimentos sin regularización